Classes:
- ontology.food_reciepe
- ontology.ingredient_set
- ontology.has_member
- ontology.is_about
- ontology.logical_xor
- ontology.ingredient_specification

In [1]:
map_table = {
		' ': '_',
        "\"": "&quot;",
        "&": "&amp;",
        "'": "&apos;",
        "<": "&lt;",
        ">": "&gt;",
        "\n": "&#xA;",
        "\r": "&#xD;",
        "\t": "&#x9;",
        "!": "&#x21;",
        "#": "&#x23;",
        "$": "&#x24;",
        "%": "&#x25;",
        "(": "&#x28;",
        ")": "&#x29;",
        "*": "&#x2A;",
        "+": "&#x2B;",
        ",": "&#x2C;",
        "-": "&#x2D;",
        ".": "&#x2E;",
        "/": "&#x2F;",
        ":": "&#x3A;",
        ";": "&#x3B;",
        "=": "&#x3D;",
        "?": "&#x3F;",
        "@": "&#x40;",
        "[": "&#x5B;",
        "\\": "&#x5C;",
        "]": "&#x5D;",
        "^": "&#x5E;",
        "_": "&#x5F;",
        "`": "&#x60;",
        "{": "&#x7B;",
        "|": "&#x7C;",
        "}": "&#x7D;",
        "~": "&#x7E;",
        "\u0085": "&#x85;",
        "\u00A0": "&#xA0;",
        "\u2028": "&#x2028;",
        "\u2029": "&#x2029;"
}

for key in map_table.keys():
    map_table[key] = '_'

In [2]:
import owlready2
import json


def load_ontology(path='../rdf/io_prototype.rdf'):
    ontology = owlready2.get_ontology(path).load()
    return ontology

def load_json(path: str):
    content = open(path)\
        .read()\
        .replace('\n', '')\
        .replace('\t', '')
    
    content = json.loads(content)
    return content

def escape_string(txt: str) -> str:
    
    for src, dst in map_table.items():
        txt = txt.replace(src, dst)
    return txt

def create_recipe_object(ontology, content):
    owl_recipe = ontology.food_reciepe(escape_string(content['title']))
    owl_recipe.is_about = [content['link']]
    return owl_recipe

def create_ingredient_set_object(ontology, content, owl_recipe):
    espaced_title = escape_string(content['title'])
    owl_id = f'__INGREDIENT_SET__{espaced_title}'
    
    owl_ingredient_set = ontology.ingredient_set(owl_id)
    owl_ingredient_set.component_of = [owl_recipe]
    return owl_ingredient_set

def create_ingredients_list(ontology, content, owl_ingredient_set):
    owl_ingredients = []
    for ingredient in content['ingredientSet']:
        # for every xor subingredients
        if isinstance(ingredient['name'], list):
            # create subingredients
            
            xor_list = []
            for name, oboId in zip(ingredient['name'], ingredient['oboId']):
                owl_ingredient = ontology.ingredient_specification(name)
                owl_food_instance = ontology.food_material(oboId)
                owl_ingredient.is_about = [owl_food_instance]
                owl_ingredient_set.has_member.append(owl_ingredient)
                
                # to be xored between pairs
                xor_list.append(owl_ingredient)
                
            # xor
            for ent1 in xor_list:
                for ent2 in xor_list:
                    if ent1 is not ent2:
                        ent1.logical_xor = [ent2]
        else:
            # create ingredient
            owl_ingredient = ontology.ingredient_specification(escape_string(ingredient['name']))
            owl_food_instance = ontology.food_material(ingredient['oboId'])
            owl_ingredient.is_about = [owl_food_instance]
            owl_ingredient_set.has_member.append(owl_ingredient)
            owl_ingredients.append(owl_ingredient)
   
    return owl_ingredients

ontology = load_ontology()

def converter(content):
    owl_recipe = create_recipe_object(ontology, content)
    owl_ingredient_set = create_ingredient_set_object(ontology, content, owl_recipe)
    owl_recipe.has_member = create_ingredients_list(ontology, content, owl_ingredient_set)

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [14]:
from tqdm import tqdm

all_content = load_json('../lexmapr_out.json')
for sample in tqdm(all_content):
    converter(sample)
    
# save ontology
file_path = "../updated_ontology.owl"
ontology.save(file=file_path)

100%|██████████| 10/10 [00:00<00:00, 1895.64it/s]
